In [14]:
from oaklib.implementations.pronto.pronto_implementation import ProntoImplementation
from oaklib.resource import OntologyResource

In [15]:
gene_ids = ['NCBIGene:6295',
 'NCBIGene:1258',
 'NCBIGene:3614',
 'NCBIGene:26121',
 'NCBIGene:7275',
 'NCBIGene:55857',
 'NCBIGene:79797',
 'NCBIGene:10594',
 'NCBIGene:64218',
 'NCBIGene:7401']

In [20]:
oi = ProntoImplementation(OntologyResource(local=False, slug='go.obo'))

In [21]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0043231 ! intracellular membrane-bounded organelle
  BFO:0000050 ! part of
    GO:0005737 ! cytoplasm


In [ ]:
labels = 

In [17]:
def enrichment_test(subjects=None, background=None, hypotheses=None, inferred_types=None, ontology_labels=None, threshold=0.05, labels=False, direction='greater'):
        """
        Performs term enrichment analysis.

        Arguments
        ---------

        subjects: string list

            Sample set. Typically a gene ID list. These are assumed to have associations

        background: string list

            Background set. If not set, uses full set of known subject IDs in the association set

        threshold: float

            p values above this are filtered out

        labels: boolean

            if true, labels for enriched classes are included in result objects

        direction: 'greater', 'less' or 'two-sided'

            default is greater - i.e. enrichment test. Use 'less' for depletion test.

        """
        if subjects is None:
            subjects = []

        subjects=set(subjects)
        bg_count = {}
        sample_count = {}
        potential_hypotheses = set()
        sample_size = len(subjects)
        for s in subjects:
            potential_hypotheses.update(inferred_types(s))
        if hypotheses is None:
            hypotheses = potential_hypotheses
        else:
            hypotheses = potential_hypotheses.intersection(hypotheses)
        logger.info("Hypotheses: {}".format(hypotheses))

        # get background counts
        # TODO: consider doing this ahead of time
        if background is None:
            background = set(subjects)
        else:
            background = set(background)

        # ensure background includes all subjects
        background.update(subjects)

        bg_size = len(background)

        for c in hypotheses:
            bg_count[c] = 0
            sample_count[c] = 0
        for s in background:
            ancs = inferred_types(s)
            for a in ancs.intersection(hypotheses):
                bg_count[a] = bg_count[a]+1
        for s in subjects:
            for a in inferred_types(s):
                if a in hypotheses:
                    sample_count[a] = sample_count[a]+1

        hypotheses = [x for x in hypotheses if bg_count[x] > 1]
        logger.info("Filtered hypotheses: {}".format(hypotheses))
        num_hypotheses = len(hypotheses)
        results = []
        for cls in hypotheses:

            # https://en.wikipedia.org/wiki/Fisher's_exact_test
            #
            #              Cls  NotCls    RowTotal
            #              ---  ------    ---
            # study/sample [a,      b]    sample_size
            # rest of ref  [c,      d]    bg_size - sample_size
            #              ---     ---
            #              nCls  nNotCls

            a = sample_count[cls]
            b = sample_size - a
            c = bg_count[cls] - a
            d = (bg_size - bg_count[cls]) - b
            #logger.debug("ABCD="+str((cls,a,b,c,d,sample_size)))
            _, p_uncorrected = sp.stats.fisher_exact( [[a, b], [c, d]], direction)
            p = p_uncorrected * num_hypotheses
            if p>1.0:
                p=1.0
            #logger.debug("P={} uncorrected={}".format(p,p_uncorrected))
            if p<threshold:
                res = {'c':cls,'p':p,'p_uncorrected':p_uncorrected}
                if labels:
                    res['n'] = ontology.label(cls)
                results.append(res)

        results = sorted(results, key=lambda x:x['p'])
        return results

In [19]:
## Run enrichment tests using all classes in ontology
enr = enrichment_test(subjects=gene_ids, threshold=0.00005, labels=True)

TypeError: 'NoneType' object is not callable

In [ ]:
## Show first 20 results
for r in enr[:20]:
        print("{:8.3g} {} {:40s}".format(r['p'],r['c'],str(r['n'])))